In [2]:
# | default_exp classes.DomoDatacenter

In [3]:
# | exporti
import asyncio
import importlib
from dataclasses import dataclass, field
from enum import Enum

import httpx

from fastcore.basics import patch_to

import domolibrary.client.DomoAuth as dmda
import domolibrary.classes.DomoDataset as dmds

import domolibrary.routes.account as account_routes
import domolibrary.routes.datacenter as datacenter_routes

In [4]:
# # | export

# class DomoEntity(Enum):
#     "enumerates the entities that can be passed to the datacenter Search API"
    
#     DATASET = "DATA_SOURCE" #dataset
#     DATAFLOW = "DATAFLOW" #  dataflow
#     PAGE = "PAGE" # page
#     CARD = "CARD" # card
#     ACCOUNT = "account" #account type


In [12]:
# | export

@dataclass
class DomoDatacenter:
    "class for quering entities in the datacenter"
    auth: dmda.DomoAuth = None


In [13]:
# # | export
# @patch_to(DomoDatacenter, cls_method=True)
# def generate_search_datacenter_body(
#     cls,
#     entity_name : str = None, # entity to search for, uses fuzzy logic
#     entity_ls: list[str] = list[DomoEntity], # list of entity types to search
#     filter_ls: [dict] = None, # filter parameters
#     combineResults: bool = False, # facet / split results
#     count: int = 100,
#     offset: int = 0,
# ):
#     body = {"query": "*" }
#     if DomoEntity.ACCOUNT in entity_ls:
#         body.update({
#             "entityList": [[DomoEntity.ACCOUNT.value]],
#         })

#         if entity_name:
#             body.update({"query": f"*{entity_name}*"})
#     else:
#         body.update({
#             "entities": [entity.value for entity in entity_ls]
#         })

#         if entity_name:
#             body.update({"field": "name_sort", "filterType": "wildcard", "query": f"*{entity_name}*"})

#     body.update({
#         "count": count, "offset": offset,
#         "filters": filter_ls or [],
#         "combineResults": combineResults,
        
#     })

#     return body


#### sample implementation of generate_search_datacenter_body

In [14]:
# DomoDatacenter.generate_search_datacenter_body(
#     entity_ls=[DomoEntity.ACCOUNT],
#     entity_name='test_rename'
# )


{'query': '*test_rename*',
 'entityList': [['account']],
 'count': 100,
 'offset': 0,
 'filters': [],
 'combineResults': False}

#### Examples of valid search strings
Notice that the account query body uses key `entityList` instead of `entities`.

Not all instances of domo will allow you to search for accounts via the Datacenter's query API, instead, consider using `DomoAccounts` to retrieve accounts filtered by name and `AccountConfig` type.


In [15]:
# s = {
#     "entityList": [["account"]],
#     "count": 100, "offset": 0,
#     "filters": [
#         {
#             "filterType": "term",
#             "field": "dataprovidername_facet",
#             "value": "Amazon Athena High Bandwidth",
#             "name": "Data Provider Type",
#             "not": False,
#             "label": "Amazon Athena High Bandwidth",
#         }
#     ],
#     "query": "**",
#     "combineResults": False,
#     "sort": {"fieldSorts": [{"field": "display_name_sort", "sortOrder": "ASC"}]},
# }

# s = {
#     "entities": ["DATASET"],
#     "count": 30, "offset": 0,
#     "filters": [
#         {
#             "filterType": "term",
#             "field": "dataprovidername_facet", "value": "DataFlow", "name": "DataFlow", "not": False
#         }],
#     "combineResults": True,
#     "query": "*", 
#     "sort": {"isRelevance": False, "fieldSorts": [{"field": "last_updated", "sortOrder": "DESC"}]}}

In [25]:
# | hide

@patch_to(DomoDatacenter, cls_method = True)
async def search_datacenter(cls, 
                            auth: dmda.DomoAuth,
                            body: dict = None,
                            session: httpx.AsyncClient = None,
                            maximum: int = None,
                            debug_api: bool = False,
                            debug_prn: bool = False) -> list:

    def arr_fn(res):
        # pprint(res.response)
        return res.response.get('searchObjects')

    def alter_maximum_fn(res):
        return res.response.get('totalResultCount')

    if not body:
        body = datacenter_routes.generate_search_datacenter_body(
            entities_list=['DATASET'],
            filters=[],
            count=1000,
            offset=0,
            combineResults=False
        )

    if debug_prn:
        print(body)

    if maximum:
        return await datacenter_routes.search_datacenter(auth=auth,
                                                            arr_fn=arr_fn,
                                                            maximum=maximum,
                                                            body=body,
                                                            session=session,
                                                            debug_api=debug_api)

    return await datacenter_routes.search_datacenter(auth=auth,
                                                        arr_fn=arr_fn,
                                                        alter_maximum_fn=alter_maximum_fn,
                                                        body=body,
                                                        session=session,
                                                        debug_api=debug_api)


#### sample implementation of search_datacenter

In [26]:
import domolibrary.client.DomoAuth as dmda
import os

token_auth = dmda.DomoTokenAuth(
    domo_instance='domo-community', domo_access_token=os.environ['DOMO_DOJO_ACCESS_TOKEN'])


ds_searchbody = DomoDatacenter.generate_search_datacenter_body(
    entity_ls=[DomoEntity.DATASET],
    entity_name='test'
)

ds_searchbody

await DomoDatacenter.search_datacenter(auth=token_auth,
                             body=ds_searchbody)


TypeError: search_datacenter() got an unexpected keyword argument 'alter_maximum_fn'

In [10]:

#     @classmethod
#     async def search_datasets(cls,
#                               auth=dmda.DomoAuth,
#                               body: dict = None,
#                               session: httpx.AsyncClient = None,
#                               maximum: int = None,
#                               debug_api=False) -> list[DomoDataset]:

#         if not body:
#             body = DomoDatacenter.generate_search_datacenter_body(
#                 entities_list=['DATASET'],
#                 filters=[],
#                 count=1000,
#                 offset=0,
#                 combineResults=False
#             )

#         if debug_api:
#             print(body)

#         json_list = await cls.search_datacenter(auth=auth,
#                                                 maximum=maximum,
#                                                 body=body,
#                                                 session=session,
#                                                 debug_api=False)

#         if json_list:
#             return await asyncio.gather(
#                 *[DomoDataset.get_from_id(id=json_obj.get('databaseId'), auth=auth, debug_api=debug_api)
#                   for json_obj in json_list]
#             )

#     @classmethod
#     async def get_lineage_upstream(cls,
#                                    auth: dmda.DomoAuth,
#                                    entity_id,
#                                    entity_type,
#                                    session: httpx.AsyncClient = None,
#                                    debug_api: bool = False,
#                                    debug_prn: bool = False):

#         import Library.DomoClasses.DomoDataflow as dmdf
#         import Library.DomoClasses.DomoDataset as dmds

#         try:
#             if not session:
#                 session = httpx.AsyncClient()
#                 is_close_session = True

#             res = await datacenter_routes.get_lineage_upstream(auth=auth,
#                                                                entity_type=entity_type,
#                                                                entity_id=entity_id,
#                                                                session=session, debug_api=debug_api)

#             if res.status == 200:
#                 obj = res.response

#                 domo_obj = []
#                 for key, item in obj.items():
#                     if item.get('type') == 'DATA_SOURCE':
#                         domo_obj.append(await dmds.DomoDataset.get_from_id(auth=auth, id=item.get('id')))

#                     if item.get('type') == 'DATAFLOW':
#                         # print(item.get('id'))
#                         domo_obj.append(await dmdf.DomoDataflow.get_from_id(auth=auth, id=item.get('id')))
#                         pass

#                 return domo_obj
#             else:
#                 return None

#         finally:
#             if is_close_session:
#                 await session.aclose()

#     @classmethod
#     async def get_accounts(cls, auth: dmda.DomoAuth,
#                            search_str: str = None,
#                            is_exact_match: bool = True,
#                            maximum: int = None,
#                            session: httpx.AsyncClient = None, debug_api: bool = False):

#         import Library.DomoClasses.DomoAccount as dma

#         json_ls = None

#         if search_str:
#             body = datacenter_routes.generate_search_datacenter_account_body(
#                 search_str=search_str, is_exact_match=is_exact_match)

#             search_ls = await cls.search_datacenter(auth=auth,
#                                                     maximum=maximum,
#                                                     body=body,
#                                                     session=session,
#                                                     debug_api=debug_api)

#             if not search_ls or len(search_ls) == 0:
#                 return None

#             json_ls = [{'id': obj.get('databaseId')} for obj in search_ls]

#         else:
#             res = await account_routes.get_accounts(auth=auth, session=session, debug_api=debug_api)

#             if debug_api:
#                 print(res)

#             if res.status != 200:
#                 return None

#             json_ls = res.response

#         domo_account_ls = await asyncio.gather(*[dma.DomoAccount.get_from_id(account_id=obj.get('id'), auth=auth) for obj in json_ls])

#         if is_exact_match and search_str:
#             return next((domo_account for domo_account in domo_account_ls if domo_account.name == search_str), None)

#         return domo_account_ls

In [11]:
#| hide
import nbdev
nbdev.nbdev_export()